# FEATURES ENGINEERING - EXAMPLES

### Importations des librairies utiles

In [ ]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.feature_selection import mutual_info_regression

print('numpy %s, pandas %s'%(np.__version__,pd.__version__))
print(os.getcwd()) 

# 0 - PREPARARATION DES DONNEES

### Creation depuis un CSV et affichage d'un Dataframe Pandas

In [ ]:
my_dataframe = pd.read_csv('Data/iris.csv')
my_dataframe

### Transformation de l'ID en index

In [ ]:
my_dataframe.set_index('id_Iris',inplace=True) #Inplace permet de conserver les modification du DataFrame
my_dataframe

### Tri selon l'index

In [ ]:
my_dataframe.sort_index(inplace=True) #Inplace permet de conserver les modification du DataFrame
my_dataframe

### Ajout de colonnes  de type catégories

In [ ]:
for taille in ["Longueur Sépale (cm)","Largeur Sépale (cm)","Longueur Pétale (cm)","Largeur Pétale (cm)"]:
    my_dataframe[taille.replace(" (cm)","")] = pd.qcut(my_dataframe[taille],3,['petit','moyen','grand'])
my_dataframe['Nom Iris'] = my_dataframe['Type Iris'].map({
    0:'Versicolor',
    1:'Setosa',
    2:'Virginica',
})
my_dataframe

In [ ]:
my_dataframe["Longueur Sépale"].value_counts()

# 1 - Exploration des données

In [ ]:
my_dataframe.sample(5)

In [ ]:
my_dataframe.info()

In [ ]:
my_dataframe.dtypes

In [ ]:
my_dataframe.describe()

# 2 - PRIORISER LES FEATURES

### Distribution des labels selon une feature

In [ ]:
display(pd.crosstab(my_dataframe["Longueur Pétale"],my_dataframe["Nom Iris"]))
print("-"*50)
display(pd.crosstab(my_dataframe["Largeur Sépale"],my_dataframe["Nom Iris"]))

### Information Mutuelle

In [ ]:
studied_features = ["Longueur Sépale","Largeur Sépale","Longueur Pétale","Largeur Pétale"]
X = my_dataframe[studied_features].copy()
for colname in X.columns:
    X[colname], _ = X[colname].factorize() # transforme des catégories en nombre entier

discrete_features = X.dtypes == "int64"

mi_scores = mutual_info_regression(
    X, 
    my_dataframe['Type Iris']    
)
mi_scores = pd.Series(mi_scores, name="MI Scores", index=X.columns)
mi_scores = mi_scores.sort_values(ascending=False)
mi_scores

In [ ]:
scores = mi_scores.sort_values(ascending=True)
width = np.arange(len(scores))
ticks = list(scores.index)
plt.barh(width, scores)
plt.yticks(width, ticks)
plt.title("Mutual Information Scores")

# 3 - Creer Des Features

### Compter des conditions (nombre de grande dimension)

In [ ]:
discrete_features = ["Longueur Sépale","Largeur Sépale","Longueur Pétale","Largeur Pétale"]
my_dataframe["nbGrand"] = (my_dataframe[discrete_features]=="grand").sum(axis=1)
display(pd.crosstab(my_dataframe["nbGrand"],my_dataframe["Nom Iris"]))
my_dataframe.sample(5, random_state=42)

### Group & Transform : Ajouter la largeur moyenne de la pétale en fonction de sa catégorie de longueur

In [ ]:
my_dataframe["Moy Largeur Pétale"] = (
    my_dataframe.groupby("Longueur Pétale")  # for each state
    ["Largeur Pétale (cm)"]                 # select the income
    .transform("mean")         # and compute its mean
)
my_dataframe

### Target Encoding

#### Sans lissage

In [ ]:
my_dataframe["Moy Type Iris"] = (
    my_dataframe.groupby("Longueur Pétale")  # for each state
    ["Type Iris"]                 # select the income
    .transform("mean")         # and compute its mean
)
my_dataframe

### Avec Lissage

In [ ]:
# Facteur de lissage = 100
coef = 150/(150 + 100)
my_dataframe["Moy Type Iris"] = coef * my_dataframe["Moy Type Iris"] + (1-coef) * my_dataframe["Type Iris"].mean()
my_dataframe["Moy Type Iris"]
my_dataframe

# 4 - Corriger ses données

### Filtrage des données

In [ ]:
my_dataframe.boxplot(fontsize=12, figsize=(18,10))

### Suppression des outliers

In [ ]:
# filtered_dataframe
exclusion_outliers = my_dataframe["Largeur Sépale (cm)"].between(2.1,4)
print("Total:",len(exclusion_outliers), "Conservés:", exclusion_outliers.sum())
filtered_dataframe =  my_dataframe[exclusion_outliers]
filtered_dataframe.boxplot(fontsize=12, figsize=(18,10))

# 5 - Compléter ses données
### Création d'un petit Dataframe avec des données manquantes dans "Longueur Sépale"

In [ ]:
# Sélection des Iris avec Longueur Sépale indéfinie
small_df_na = my_dataframe[my_dataframe['Longueur Sépale (cm)'].isna()]
# Sélection aléatoire des Iris avec Longueur Sépale définie
small_df_rnd = my_dataframe[~my_dataframe['Longueur Sépale (cm)'].isna()].sample(20, random_state=42)
my_dataframe[my_dataframe['Longueur Sépale (cm)'].isna()]
small_df = pd.concat([small_df_na,small_df_rnd]).sort_index()
small_df

### Suppression des lignes avec des données manquantes

In [ ]:
small_df[~small_df.isna().any(axis=1)]

### Suppression d'une colonne

In [ ]:
small_df.drop('Longueur Sépale (cm)', axis=1)

### Remplacement par la moyenne

In [ ]:
fulfilled_small_df=small_df.copy()
fulfilled_small_df["Longueur Sépale (cm)"].fillna(small_df["Longueur Sépale (cm)"].mean(),inplace=True)
fulfilled_small_df

### Régression linéaire entre Longueur Sépale et Longueur Pétale

In [ ]:
filter_not_nan = ~(small_df['Longueur Sépale (cm)'].isnull() | small_df['Longueur Pétale (cm)'].isnull())
x = small_df.loc[filter_not_nan,'Longueur Pétale (cm)'].values.reshape(-1, 1) # Mise au format pour le fit, car plusieurs variables possibles
print(x)
y = small_df.loc[filter_not_nan,'Longueur Sépale (cm)']
model = LinearRegression()

model.fit(x,y)
y_predict = model.predict(x)

plt.scatter(x, y)
plt.plot(x, y_predict, color='g')
plt.show()

In [ ]:
x_reg = small_df.loc[small_df['Longueur Sépale (cm)'].isnull() ,'Longueur Pétale (cm)'].values.reshape(-1, 1)
y_reg = model.predict(x_reg)

### Affichage des points calculé par régression linéaire

In [ ]:
plt.scatter(x, y)
plt.scatter(x_reg, y_reg, color='r')
plt.plot(x, y_predict, color='g')
plt.show()
fulfilled_small_df.loc[small_df['Longueur Sépale (cm)'].isnull() ,'Longueur Sépale (cm)'] = y_reg
fulfilled_small_df